In [13]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
import glob
import os

# --- PARAMETERS ---
PSC_MERGE_THRESHOLD = 30   # pixels: merge PSCs within this horizontal distance

def preprocess_for_psc(img_path, pad=10):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bw = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    h, w = bw.shape
    canvas = np.zeros((h + 2 * pad, w + 2 * pad), dtype=np.uint8)
    canvas[pad:pad + h, pad:pad + w] = bw
    return canvas


# --- 1) LOAD AND THRESHOLD ---
def load_and_binarize(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    # if color, convert to gray
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # adaptive or Otsu threshold
    _, bw = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # now text is white (255), background is black (0)
    return bw

# --- 2) THIN/SKELETONIZE ---
def thin_image(bw):
    # skeletonize from skimage expects boolean
    skeleton = skeletonize(bw > 0)
    return (skeleton.astype(np.uint8) * 255)

# --- 3) NOISE REMOVAL ---
def remove_noise(skel, min_component=30):
    # remove small connected components
    nb_components, labels, stats, _ = cv2.connectedComponentsWithStats(skel, connectivity=8)
    cleaned = np.zeros_like(skel)
    for i in range(1, nb_components):
        if stats[i, cv2.CC_STAT_AREA] >= min_component:
            cleaned[labels == i] = 255
    return cleaned

# --- 4) FIND POTENTIAL SEGMENTATION COLUMNS (PSCs) ---
def find_psc_columns(skel):
    h, w = skel.shape
    # sum of foreground pixels in each column
    col_sums = (skel > 0).sum(axis=0)
    # PSC = columns where sum is 0 or 1
    pscs = np.where(col_sums<2)[0]
    return pscs

# --- 5) MERGE PSCs INTO FINAL SEGMENTATION COLUMNS ---
def merge_pscs(pscs, threshold=20, min_dist_from_edge=10, min_seg_width=20, img_width=None):
    if not pscs.any() or img_width is None:
        return []

    merged = []
    prev = pscs[0]
    for current in pscs[1:]:
        if current - prev > threshold:
            merged.append(prev)
            prev = current
        else:
            prev = (prev + current) // 2
    merged.append(prev)

    # remove cuts too close to image edge
    merged = [x for x in merged if min_dist_from_edge < x < img_width - min_dist_from_edge]

    # remove segments too narrow
    cleaned = []
    last = 0
    for x in merged:
        if x - last >= min_seg_width:
            cleaned.append(x)
            last = x
    return cleaned


# --- 6) APPLY SEGMENTATION AND SAVE OUTPUTS ---
def segment_word(path_in, out_dir):
    base = os.path.splitext(os.path.basename(path_in))[0]
    
    # Use padded, binarized word image
    bw = preprocess_for_psc(path_in, pad=10)
    bw = cv2.resize(bw, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LINEAR)

    
    skel = thin_image(bw)
    skel = remove_noise(skel)
    pscs = find_psc_columns(skel, img_name=base)
    seg_cols = merge_pscs(pscs, img_width=bw.shape[1])


    # visualize PSCs overlaid in red (raw) and green (final)
    debug = cv2.cvtColor(bw, cv2.COLOR_GRAY2BGR)
    for x in pscs:
        cv2.line(debug, (x, 0), (x, bw.shape[0] - 1), (0, 0, 255), 1)
    for x in seg_cols:
        cv2.line(debug, (x, 0), (x, bw.shape[0] - 1), (0, 255, 0), 1)
    os.makedirs(out_dir, exist_ok=True)
    cv2.imwrite(os.path.join(out_dir, f"{base}_debug.png"), debug)

    # segment characters
    xs = [0] + seg_cols + [bw.shape[1]]
    for i in range(len(xs) - 1):
        x0, x1 = xs[i], xs[i + 1]
        char = bw[:, x0:x1]
        if np.sum(char) == 0:
            continue
        cv2.imwrite(os.path.join(out_dir, f"{base}_char_{i:02d}.png"), char)
    
def save_column_heatmap(col_sums, img_name):
    df = pd.DataFrame({'column_index': np.arange(len(col_sums)), 'pixel_count': col_sums})
    df.to_csv(f"{img_name}_column_heatmap.csv", index=False)

        
def find_psc_columns(skel, img_name="unknown"):
    col_sums = (skel > 0).sum(axis=0)
    # Save the raw column values to a text file for inspection
    np.savetxt(f"{img_name}_column_sums.txt", col_sums, fmt="%d")
    
    # Print a few key stats
    print(f"[{img_name}] Column sum (foreground pixel count per column):")
    print("Index : Value")
    for i, v in enumerate(col_sums):
        print(f"{i:03d} : {v}")
    
    # Select PSCs: columns with 0 or 1 pixel
    pscs = np.where(col_sums<10)[0]
    return pscs

        
# --- 7) BATCH PROCESS ---
if __name__ == "__main__":
    inp_folder  = "cropped_words"
    out_folder  = "words_out"
    for img_path in glob.glob(os.path.join(inp_folder, "*.png")) + \
                    glob.glob(os.path.join(inp_folder, "*.bmp")) + \
                    glob.glob(os.path.join(inp_folder, "*.jpg")):
        segment_word(img_path, out_folder)
    print("Done.")


[line01_word000] Column sum (foreground pixel count per column):
Index : Value
000 : 0
001 : 0
002 : 0
003 : 0
004 : 0
005 : 0
006 : 0
007 : 0
008 : 0
009 : 0
010 : 0
011 : 0
012 : 0
013 : 0
014 : 0
015 : 0
016 : 0
017 : 0
018 : 0
019 : 0
020 : 0
021 : 0
022 : 0
023 : 0
024 : 0
025 : 0
026 : 20
027 : 3
028 : 2
029 : 3
030 : 3
031 : 2
032 : 2
033 : 48
034 : 24
035 : 7
036 : 3
037 : 3
038 : 3
039 : 3
040 : 3
041 : 3
042 : 3
043 : 3
044 : 4
045 : 5
046 : 2
047 : 2
048 : 2
049 : 2
050 : 2
051 : 2
052 : 2
053 : 2
054 : 2
055 : 2
056 : 2
057 : 3
058 : 4
059 : 4
060 : 4
061 : 4
062 : 4
063 : 4
064 : 6
065 : 6
066 : 33
067 : 26
068 : 5
069 : 9
070 : 17
071 : 11
072 : 12
073 : 11
074 : 11
075 : 10
076 : 10
077 : 9
078 : 8
079 : 9
080 : 10
081 : 7
082 : 7
083 : 7
084 : 8
085 : 8
086 : 8
087 : 8
088 : 8
089 : 9
090 : 8
091 : 9
092 : 8
093 : 9
094 : 9
095 : 10
096 : 9
097 : 9
098 : 9
099 : 9
100 : 38
101 : 12
102 : 16
103 : 9
104 : 9
105 : 14
106 : 14
107 : 11
108 : 20
109 : 7
110 : 7
111 : 7
112 

[line02_word005] Column sum (foreground pixel count per column):
Index : Value
000 : 0
001 : 0
002 : 0
003 : 0
004 : 0
005 : 0
006 : 0
007 : 0
008 : 0
009 : 0
010 : 0
011 : 0
012 : 0
013 : 0
014 : 0
015 : 0
016 : 0
017 : 0
018 : 0
019 : 0
020 : 0
021 : 0
022 : 0
023 : 0
024 : 0
025 : 36
026 : 17
027 : 11
028 : 11
029 : 14
030 : 15
031 : 17
032 : 19
033 : 18
034 : 18
035 : 21
036 : 21
037 : 21
038 : 19
039 : 22
040 : 20
041 : 23
042 : 21
043 : 22
044 : 21
045 : 22
046 : 21
047 : 21
048 : 20
049 : 20
050 : 19
051 : 20
052 : 20
053 : 19
054 : 19
055 : 22
056 : 19
057 : 18
058 : 19
059 : 19
060 : 20
061 : 20
062 : 18
063 : 18
064 : 20
065 : 14
066 : 14
067 : 15
068 : 13
069 : 14
070 : 13
071 : 14
072 : 13
073 : 15
074 : 13
075 : 13
076 : 12
077 : 13
078 : 12
079 : 11
080 : 13
081 : 18
082 : 17
083 : 11
084 : 16
085 : 14
086 : 10
087 : 11
088 : 10
089 : 10
090 : 10
091 : 15
092 : 10
093 : 10
094 : 9
095 : 7
096 : 7
097 : 6
098 : 9
099 : 7
100 : 5
101 : 8
102 : 6
103 : 6
104 : 7
105 : 8
106 

[line04_word015] Column sum (foreground pixel count per column):
Index : Value
000 : 0
001 : 0
002 : 0
003 : 0
004 : 0
005 : 0
006 : 0
007 : 0
008 : 0
009 : 0
010 : 0
011 : 0
012 : 0
013 : 0
014 : 0
015 : 0
016 : 0
017 : 0
018 : 0
019 : 0
020 : 0
021 : 0
022 : 0
023 : 0
024 : 0
025 : 24
026 : 14
027 : 5
028 : 13
029 : 6
030 : 6
031 : 6
032 : 5
033 : 5
034 : 5
035 : 5
036 : 5
037 : 6
038 : 5
039 : 6
040 : 5
041 : 5
042 : 5
043 : 5
044 : 11
045 : 13
046 : 4
047 : 4
048 : 3
049 : 3
050 : 3
051 : 3
052 : 3
053 : 3
054 : 3
055 : 3
056 : 4
057 : 4
058 : 4
059 : 6
060 : 8
061 : 14
062 : 8
063 : 9
064 : 9
065 : 8
066 : 14
067 : 16
068 : 9
069 : 8
070 : 9
071 : 8
072 : 9
073 : 8
074 : 8
075 : 8
076 : 7
077 : 7
078 : 9
079 : 7
080 : 9
081 : 7
082 : 23
083 : 8
084 : 5
085 : 5
086 : 5
087 : 7
088 : 6
089 : 6
090 : 6
091 : 8
092 : 10
093 : 18
094 : 14
095 : 8
096 : 11
097 : 13
098 : 15
099 : 9
100 : 8
101 : 8
102 : 7
103 : 8
104 : 7
105 : 7
106 : 7
107 : 5
108 : 5
109 : 5
110 : 5
111 : 4
112 : 8
11